In [1]:
import pandas as pd
import re
import itertools

# Load CSV file
file_path = "components_voxel_from_min_10msizeVER02.csv"  # ✅ Update this if needed
df = pd.read_csv(file_path)

# Column names
voxel_column = 'Voxel'
component_column = df.columns[0] if df.columns[0] != voxel_column else df.columns[1]

# Function to parse Vx(x,y,z)
def parse_voxel_name(voxel_name):
    match = re.match(r"Vx\((\d+),(\d+),(\d+)\)", voxel_name)
    if match:
        return int(match.group(1)), int(match.group(2)), int(match.group(3))
    return None, None, None

# Parse voxel coordinates
df[['x', 'y', 'z']] = df[voxel_column].apply(lambda v: pd.Series(parse_voxel_name(v)))

# Compute actual voxel component counts
voxel_counts = df.groupby(['x', 'y', 'z'])[component_column].count().reset_index()
voxel_counts.columns = ['x', 'y', 'z', 'ComponentCount']

# Determine full range of x, y, z
max_x, max_y, max_z = df['x'].max(), df['y'].max(), df['z'].max()
all_voxels = pd.DataFrame(list(itertools.product(range(max_x + 1), range(max_y + 1), range(max_z + 1))), columns=['x', 'y', 'z'])

# Merge actual with full set
full_voxel_data = pd.merge(all_voxels, voxel_counts, on=['x', 'y', 'z'], how='left')
full_voxel_data['ComponentCount'] = full_voxel_data['ComponentCount'].fillna(0).astype(int)

# Generate voxel names like Vx(x,y,z)
full_voxel_data['Voxel'] = full_voxel_data.apply(lambda row: f"Vx({row['x']},{row['y']},{row['z']})", axis=1)

# Reorder columns
final_df = full_voxel_data[['Voxel', 'x', 'y', 'z', 'ComponentCount']]

# Print the result
print(final_df)

# Optional: Save to CSV
final_df.to_csv("all_voxel_component_counts.csv", index=False)


         Voxel  x  y  z  ComponentCount
0    Vx(0,0,0)  0  0  0               0
1    Vx(0,0,1)  0  0  1               1
2    Vx(0,0,2)  0  0  2               0
3    Vx(0,0,3)  0  0  3               0
4    Vx(0,0,4)  0  0  4               0
..         ... .. .. ..             ...
275  Vx(6,7,0)  6  7  0               0
276  Vx(6,7,1)  6  7  1               0
277  Vx(6,7,2)  6  7  2               0
278  Vx(6,7,3)  6  7  3               0
279  Vx(6,7,4)  6  7  4               0

[280 rows x 5 columns]


In [2]:
import pandas as pd
import plotly.graph_objects as go

# --- 1) Load CSV and extract voxel indices
df = pd.read_csv("components_voxel_from_min_10msizeVER02.csv")

# Each "Voxel" is like "Vx(x,y,z)". Strip “Vx(” and “)” and split on commas.
df[['VoxelX', 'VoxelY', 'VoxelZ']] = (
    df['Voxel']
      .str.replace(r"^Vx\(", "", regex=True)   # remove leading "Vx("
      .str.replace(r"\)$", "",   regex=True)   # remove trailing ")"
      .str.split(",", expand=True)             # split "x,y,z" into three strings
      .astype(int)                             # convert to int
)

# --- 2) Group components by their voxel coordinates
grouped = df.groupby(['VoxelX','VoxelY','VoxelZ'])

# Build a list of unique voxel coordinates and also remember the original "Voxel" string
voxel_list = []
for (vx, vy, vz), subdf in grouped:
    # original voxel string is the same for all rows in this group
    voxel_name = subdf['Voxel'].iloc[0]  # e.g. "Vx(10,10,0)"
    voxel_list.append((vx, vy, vz, voxel_name))

# --- 3) Helpers to create a wireframe cube (5×5×5 m) at lower‐corner (x,y,z)
def cube_corners(x, y, z, size=5):
    """
    Return the 8 corner points of a cube of edge length `size`,
    whose lower‐corner (min X, min Y, min Z) sits at (x,y,z).
    """
    x0, x1 = x, x + size
    y0, y1 = y, y + size
    z0, z1 = z, z + size

    return [
        (x0, y0, z0),
        (x1, y0, z0),
        (x1, y1, z0),
        (x0, y1, z0),
        (x0, y0, z1),
        (x1, y0, z1),
        (x1, y1, z1),
        (x0, y1, z1),
    ]

def cube_edges_xyz(corners):
    """
    Given 8 corner‐tuples in the order returned by cube_corners(),
    return three lists (x_edges, y_edges, z_edges) that trace out
    the 12 edges, inserting None between segments.
    """
    edge_pairs = [
        (0,1), (1,2), (2,3), (3,0),  # bottom loop
        (4,5), (5,6), (6,7), (7,4),  # top loop
        (0,4), (1,5), (2,6), (3,7),  # vertical edges
    ]
    x_lines, y_lines, z_lines = [], [], []
    for i0, i1 in edge_pairs:
        x0,y0,z0 = corners[i0]
        x1,y1,z1 = corners[i1]
        x_lines += [x0, x1, None]
        y_lines += [y0, y1, None]
        z_lines += [z0, z1, None]
    return x_lines, y_lines, z_lines

# --- 4) Build the wireframe edges and also record each cube’s center + hover‐text + label
all_x, all_y, all_z = [], [], []
centers_x, centers_y, centers_z = [], [], []
hover_texts, voxel_labels = [], []

for (vx, vy, vz, vname) in voxel_list:
    # Convert voxel indices to meters
    x_m = vx * 5
    y_m = vy * 5
    z_m = vz * 5

    # 4a) Build the corner points and edges
    corners = cube_corners(x_m, y_m, z_m, size=5)
    x_e, y_e, z_e = cube_edges_xyz(corners)
    all_x += x_e
    all_y += y_e
    all_z += z_e

    # 4b) Compute center‐point for hover interaction
    cx = x_m + 2.5
    cy = y_m + 2.5
    cz = z_m + 2.5
    centers_x.append(cx)
    centers_y.append(cy)
    centers_z.append(cz)

    # 4c) Build a hovertext listing all components in that voxel
    subdf = grouped.get_group((vx, vy, vz))
    lines = [f"{gid} / {nm}" for gid, nm in zip(subdf['GlobalId'], subdf['Name'])]
    hover_texts.append("<br>".join(lines))

    # 4d) Store the voxel’s name (e.g. "Vx(10,10,0)") for later
    voxel_labels.append(vname)

# Create a single Scatter3d for all edges (wireframe)
edge_trace = go.Scatter3d(
    x=all_x,
    y=all_y,
    z=all_z,
    mode='lines',
    line=dict(color='steelblue', width=2),
    hoverinfo='none'  # no hover on the edges themselves
)

# Create a Scatter3d of small markers at each cube‐center, to show hover‐text
center_trace = go.Scatter3d(
    x=centers_x,
    y=centers_y,
    z=centers_z,
    mode='markers',
    marker=dict(size=4, color='crimson', opacity=0.6),
    hovertext=hover_texts,
    hoverinfo='text'
)

# Create another Scatter3d whose sole purpose is to display each voxel name as 3D text
# We shift the z‐coordinate up by a small amount (e.g. +0.2) so the text floats just above center
label_trace = go.Scatter3d(
    x=centers_x,
    y=centers_y,
    z=[z + 0.2 for z in centers_z],   # slightly above the center
    mode='text',
    text=voxel_labels,                # e.g. ["Vx(10,10,0)", "Vx(10,9,0)", ...]
    textfont=dict(size=10, color='black'),
    hoverinfo='none'  # we don’t need additional hover on the text itself
)

# --- 5) Combine everything into one Figure
fig = go.Figure(data=[edge_trace, center_trace, label_trace])

fig.update_layout(
    title="Wireframe Voxels (10×10×10 m) with Voxel Names and Component Hover",
    scene=dict(
        xaxis=dict(title='X (m)', gridcolor='lightgray'),
        yaxis=dict(title='Y (m)', gridcolor='lightgray'),
        zaxis=dict(title='Z (m)', gridcolor='lightgray'),
        aspectmode='data'  # keep equal aspect ratio in X, Y, Z
    ),
    margin=dict(l=0, r=0, b=0, t=40)
)

#fig.show()

# --- 6) (Optional) Export as a standalone HTML to share
output_filename = "voxels_with_labels.html"
fig.write_html(output_filename, auto_open=False)
print(f"Interactive HTML saved as: {output_filename}")


Interactive HTML saved as: voxels_with_labels.html


In [3]:
import pandas as pd
import plotly.graph_objects as go

# --- 1) Load CSV and extract voxel indices
df = pd.read_csv("components_voxel_from_min_10msizeVER02.csv")

# Each "Voxel" is like "Vx(x,y,z)". Strip “Vx(” and “)” and split on commas.
df[['VoxelX', 'VoxelY', 'VoxelZ']] = (
    df['Voxel']
      .str.replace(r"^Vx\(", "", regex=True)   # remove leading "Vx("
      .str.replace(r"\)$", "",   regex=True)   # remove trailing ")"
      .str.split(",", expand=True)             # split "x,y,z" into three strings
      .astype(int)                             # convert to int
)

# --- 2) Group components by their voxel coordinates
grouped = df.groupby(['VoxelX','VoxelY','VoxelZ'])

# Build a list of unique voxel coordinates and also remember the original "Voxel" string
voxel_list = []
for (vx, vy, vz), subdf in grouped:
    voxel_name = subdf['Voxel'].iloc[0]  # e.g. "Vx(10,10,0)"
    voxel_list.append((vx, vy, vz, voxel_name))

# --- 3) Helpers to create a wireframe cube (5×5×5 m) at lower‐corner (x,y,z)
def cube_corners(x, y, z, size=5):
    x0, x1 = x, x + size
    y0, y1 = y, y + size
    z0, z1 = z, z + size
    return [
        (x0, y0, z0),
        (x1, y0, z0),
        (x1, y1, z0),
        (x0, y1, z0),
        (x0, y0, z1),
        (x1, y0, z1),
        (x1, y1, z1),
        (x0, y1, z1),
    ]

def cube_edges_xyz(corners):
    edge_pairs = [
        (0,1), (1,2), (2,3), (3,0),  # bottom loop
        (4,5), (5,6), (6,7), (7,4),  # top loop
        (0,4), (1,5), (2,6), (3,7),  # vertical edges
    ]
    x_lines, y_lines, z_lines = [], [], []
    for i0, i1 in edge_pairs:
        x0,y0,z0 = corners[i0]
        x1,y1,z1 = corners[i1]
        x_lines += [x0, x1, None]
        y_lines += [y0, y1, None]
        z_lines += [z0, z1, None]
    return x_lines, y_lines, z_lines

# --- 4) Build the wireframe edges and also record each cube’s center + hover‐text + label
all_x, all_y, all_z = [], [], []
centers_x, centers_y, centers_z = [], [], []
hover_texts, voxel_labels = [], []
component_counts = []  # For color scale

for (vx, vy, vz, vname) in voxel_list:
    x_m = vx * 5
    y_m = vy * 5
    z_m = vz * 5

    corners = cube_corners(x_m, y_m, z_m, size=5)
    x_e, y_e, z_e = cube_edges_xyz(corners)
    all_x += x_e
    all_y += y_e
    all_z += z_e

    cx = x_m + 2.5
    cy = y_m + 2.5
    cz = z_m + 2.5
    centers_x.append(cx)
    centers_y.append(cy)
    centers_z.append(cz)

    subdf = grouped.get_group((vx, vy, vz))
    lines = [f"{gid} / {nm}" for gid, nm in zip(subdf['GlobalId'], subdf['Name'])]
    hover_texts.append("<br>".join(lines))
    voxel_labels.append(vname)
    component_counts.append(len(subdf))  # Number of components in the voxel

# --- 5) Create Plotly Traces

# Wireframe edges
edge_trace = go.Scatter3d(
    x=all_x,
    y=all_y,
    z=all_z,
    mode='lines',
    line=dict(color='steelblue', width=2),
    hoverinfo='none'
)

# Center points colored by component count
center_trace = go.Scatter3d(
    x=centers_x,
    y=centers_y,
    z=centers_z,
    mode='markers',
    marker=dict(
        size=6,
        color=component_counts,
        colorscale='Viridis',
        opacity=0.5,
        colorbar=dict(title='Components per Voxel')
    ),
    hovertext=hover_texts,
    hoverinfo='text'
)

# Voxel name labels
label_trace = go.Scatter3d(
    x=centers_x,
    y=centers_y,
    z=[z + 0.2 for z in centers_z],
    mode='text',
    text=voxel_labels,
    textfont=dict(size=10, color='black'),
    hoverinfo='none'
)

# --- 6) Create Figure
fig = go.Figure(data=[edge_trace, center_trace, label_trace])

fig.update_layout(
    title="Voxel Grid Colored by Number of Components (Transparent View)",
    scene=dict(
        xaxis=dict(title='X (m)', gridcolor='lightgray'),
        yaxis=dict(title='Y (m)', gridcolor='lightgray'),
        zaxis=dict(title='Z (m)', gridcolor='lightgray'),
        aspectmode='data'
    ),
    margin=dict(l=0, r=0, b=0, t=40)
)

# Optional: Export to HTML
output_filename = "voxels_colored_by_components.html"
fig.write_html(output_filename, auto_open=False)
print(f"Interactive HTML saved as: {output_filename}")


Interactive HTML saved as: voxels_colored_by_components.html


In [5]:
import pandas as pd
import plotly.graph_objects as go

# --- 1) Load CSV and extract voxel indices
df = pd.read_csv("components_voxel_from_min_10msizeVER02.csv")

# Parse voxel indices from strings like "Vx(1,2,3)"
df[['VoxelX', 'VoxelY', 'VoxelZ']] = (
    df['Voxel']
      .str.replace(r"^Vx\(", "", regex=True)
      .str.replace(r"\)$", "", regex=True)
      .str.split(",", expand=True)
      .astype(int)
)

# --- 2) Group by voxel coordinates
grouped = df.groupby(['VoxelX', 'VoxelY', 'VoxelZ'])

# --- 3) Helper to build cube geometry for Mesh3d
def create_voxel_mesh(x0, y0, z0, size=5):
    x1, y1, z1 = x0 + size, y0 + size, z0 + size
    vertices = [
        (x0, y0, z0), (x1, y0, z0), (x1, y1, z0), (x0, y1, z0),  # bottom
        (x0, y0, z1), (x1, y0, z1), (x1, y1, z1), (x0, y1, z1),  # top
    ]
    faces = [
        (0, 1, 2), (0, 2, 3),  # bottom
        (4, 5, 6), (4, 6, 7),  # top
        (0, 1, 5), (0, 5, 4),  # front
        (1, 2, 6), (1, 6, 5),  # right
        (2, 3, 7), (2, 7, 6),  # back
        (3, 0, 4), (3, 4, 7),  # left
    ]
    return vertices, faces

# --- 4) Build mesh for each voxel
all_vertices = []
all_i, all_j, all_k = [], [], []
face_intensity = []
hover_texts = []

vertex_offset = 0

for (vx, vy, vz), group in grouped:
    x0, y0, z0 = vx * 5, vy * 5, vz * 5
    vertices, faces = create_voxel_mesh(x0, y0, z0, size=5)
    
    all_vertices.extend(vertices)
    
    for face in faces:
        i, j, k = [vertex_offset + idx for idx in face]
        all_i.append(i)
        all_j.append(j)
        all_k.append(k)
    
    count = len(group)
    face_intensity.extend([count] * len(faces))  # one value per triangle

    lines = [f"{gid} / {name}" for gid, name in zip(group['GlobalId'], group['Name'])]
    hover_texts.extend([("<br>".join(lines))] * len(faces))  # repeat for each face

    vertex_offset += 8

# --- 5) Unpack vertex coordinates
x, y, z = zip(*all_vertices)

# --- 6) Create Mesh3d trace
fig = go.Figure()

fig.add_trace(go.Mesh3d(
    x=x, y=y, z=z,
    i=all_i, j=all_j, k=all_k,
    intensity=face_intensity,
    colorscale='Viridis',
    opacity=0.3,  # Transparent cubes
    flatshading=True,
    hoverinfo='text',
    hovertext=hover_texts,
    showscale=True,
    colorbar=dict(title="Component Count")
))

fig.update_layout(
    title="Transparent Voxel Cubes Colored by Number of Components",
    scene=dict(
        xaxis=dict(title='X (m)', gridcolor='lightgray'),
        yaxis=dict(title='Y (m)', gridcolor='lightgray'),
        zaxis=dict(title='Z (m)', gridcolor='lightgray'),
        aspectmode='data'
    ),
    margin=dict(l=0, r=0, b=0, t=40)
)

# --- 7) Export to HTML
output_filename = "transparent_voxels_by_components.html"
fig.write_html(output_filename, auto_open=False)
print(f"Interactive HTML saved as: {output_filename}")


Interactive HTML saved as: transparent_voxels_by_components.html


In [6]:
import pandas as pd
import plotly.graph_objects as go

# --- 1) Load CSV and extract voxel indices
df = pd.read_csv("components_voxel_from_min_10msizeVER02.csv")

# Parse voxel indices from strings like "Vx(1,2,3)"
df[['VoxelX', 'VoxelY', 'VoxelZ']] = (
    df['Voxel']
      .str.replace(r"^Vx\(", "", regex=True)
      .str.replace(r"\)$", "", regex=True)
      .str.split(",", expand=True)
      .astype(int)
)

# --- 2) Group by voxel coordinates
grouped = df.groupby(['VoxelX', 'VoxelY', 'VoxelZ'])

# --- 3) Helpers
def create_voxel_mesh(x0, y0, z0, size=5):
    """Return 8 cube vertices and 12 triangular faces (Mesh3d)"""
    x1, y1, z1 = x0 + size, y0 + size, z0 + size
    vertices = [
        (x0, y0, z0), (x1, y0, z0), (x1, y1, z0), (x0, y1, z0),  # bottom
        (x0, y0, z1), (x1, y0, z1), (x1, y1, z1), (x0, y1, z1),  # top
    ]
    faces = [
        (0, 1, 2), (0, 2, 3),  # bottom
        (4, 5, 6), (4, 6, 7),  # top
        (0, 1, 5), (0, 5, 4),  # front
        (1, 2, 6), (1, 6, 5),  # right
        (2, 3, 7), (2, 7, 6),  # back
        (3, 0, 4), (3, 4, 7),  # left
    ]
    return vertices, faces

def cube_edges(x0, y0, z0, size=5):
    """Return x, y, z lists for the 12 cube edges as line segments"""
    x1, y1, z1 = x0 + size, y0 + size, z0 + size
    corners = [
        (x0, y0, z0), (x1, y0, z0), (x1, y1, z0), (x0, y1, z0),
        (x0, y0, z1), (x1, y0, z1), (x1, y1, z1), (x0, y1, z1),
    ]
    edges = [
        (0, 1), (1, 2), (2, 3), (3, 0),  # bottom
        (4, 5), (5, 6), (6, 7), (7, 4),  # top
        (0, 4), (1, 5), (2, 6), (3, 7),  # verticals
    ]
    xs, ys, zs = [], [], []
    for i, j in edges:
        for coord in [corners[i], corners[j], (None, None, None)]:
            x, y, z = coord
            xs.append(x)
            ys.append(y)
            zs.append(z)
    return xs, ys, zs

# --- 4) Build geometry
all_vertices = []
all_i, all_j, all_k = [], [], []
face_intensity = []
hover_texts = []
wire_x, wire_y, wire_z = [], [], []
vertex_offset = 0

for (vx, vy, vz), group in grouped:
    x0, y0, z0 = vx * 5, vy * 5, vz * 5
    vertices, faces = create_voxel_mesh(x0, y0, z0, size=5)
    all_vertices.extend(vertices)
    for face in faces:
        i, j, k = [vertex_offset + idx for idx in face]
        all_i.append(i)
        all_j.append(j)
        all_k.append(k)
    count = len(group)
    face_intensity.extend([count] * len(faces))
    lines = [f"{gid} / {name}" for gid, name in zip(group['GlobalId'], group['Name'])]
    hover_texts.extend(["<br>".join(lines)] * len(faces))
    vertex_offset += 8

    # Add wireframe
    xw, yw, zw = cube_edges(x0, y0, z0, size=5)
    wire_x += xw
    wire_y += yw
    wire_z += zw

# --- 5) Unpack vertex coordinates
x, y, z = zip(*all_vertices)

# --- 6) Plot
fig = go.Figure()

# Mesh3d for transparent cubes
fig.add_trace(go.Mesh3d(
    x=x, y=y, z=z,
    i=all_i, j=all_j, k=all_k,
    intensity=face_intensity,
    colorscale='Viridis',
    opacity=0.3,  # Transparent
    flatshading=True,
    hoverinfo='text',
    hovertext=hover_texts,
    showscale=True,
    colorbar=dict(title="Components per Voxel")
))

# Wireframe lines
fig.add_trace(go.Scatter3d(
    x=wire_x, y=wire_y, z=wire_z,
    mode='lines',
    line=dict(color='black', width=1),
    hoverinfo='none',
    name='Voxel Edges'
))

fig.update_layout(
    title="Transparent Voxel Cubes with Colored Fill and Wireframe Edges",
    scene=dict(
        xaxis=dict(title='X (m)', gridcolor='lightgray'),
        yaxis=dict(title='Y (m)', gridcolor='lightgray'),
        zaxis=dict(title='Z (m)', gridcolor='lightgray'),
        aspectmode='data'
    ),
    margin=dict(l=0, r=0, b=0, t=40)
)

# --- 7) Save as interactive HTML
output_filename = "transparent_voxels_with_edges.html"
fig.write_html(output_filename, auto_open=False)
print(f"Interactive HTML saved as: {output_filename}")


Interactive HTML saved as: transparent_voxels_with_edges.html
